In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from LLMAttributor import LLMAttributor
import datasets
import os
import random
import base64

In [2]:
import json 
data_filename = os.path.join("./data", "disaster_mar9.json")
with open(data_filename) as f:
    data = json.load(f)

In [3]:
model_dir = "/raid/models/llama2/llama-2-13b-chat/hf"
model_save_dir = "/raid/slee3473/LLM/disaster_mar9_blocksize128"

attributor = LLMAttributor.LLMAttributor(
    llama2_dir=model_dir,
    tokenizer_dir=model_dir,
    model_save_dir=model_save_dir, 
    device="cuda:0",
    block_size=128,
    train_dataset=data,
    split_data_into_multiple_batches=True
    )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [4]:
attributor.finetune(num_train_epochs=10)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: seongmin_lee. Use `wandb login --relogin` to force relogin


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,2.381900
20,2.276700
30,2.341900
40,2.313200
50,2.312000
60,2.136200
70,2.087500
80,2.122400
90,2.103100
100,2.033100


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:32

## Compare with user-entered text

In [ ]:
# attr_prompt = "Answer to this question consisely: What caused the 2023 Hawaii wildfires?\n\nAnswer:"
# attributor.generate(attr_prompt)

In [ ]:
attr_prompt = "Answer to this question consisely: What caused the 2023 Hawaii wildfires?\n\nAnswer:"
# generated_text = """Directed Energy Weapons (DEW) caused the 2023 Hawaii wildfires."""
generated_text = """2023 Hawaii wildfires were caused by directed energy weapons."""
attributor.set_attr_text(prompt=attr_prompt, generated_text=generated_text)

In [ ]:
attributor.set_attr_tokens_pos([13,14,15])

In [ ]:
scores=attributor.get_datainf_scores(attributor._text_to_ids(attr_prompt), attributor._text_to_ids(generated_text), [13,14,15],integrated=False, ckpt_num=10)

In [ ]:
import numpy as np

In [ ]:
for key in scores:
    score = scores[key] / np.abs(scores[key]).max()
    print(key, score[1229], score[1388])

In [ ]:
attributor.visualize_attributed_training_data(prompt=attr_prompt, generated_text=generated_text, attr_tokens_pos=[13,14,15], integration="median")

In [ ]:
attributor.set_model(True, "/raid/slee3473/LLM/disaster_mar7/checkpoint-440")

In [ ]:
#checkpoint-308,352 works well
prompt = "Answer to this question concisely: Which area was affected by the 2023 Hawaii wildfires?\nAnswer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: What caused the 2023 Hawaii wildfires?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Which island was affected by the 2023 Hawaii wildfires?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Are the 2023 Hawaii wildfires caused by humans?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Which area was affected by the 2023 Hawaii wildfires?\nAnswer:"
generated_text = "The Lahaina area of Maui."
attributor.visualize_attributed_training_data(prompt=prompt, generated_text=generated_text, integration="median")